In [46]:
#define Network
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tqdm import tqdm
def base_model():
    inputs = tf.keras.Input(shape=(784,),name = "inputs")
    x = tf.keras.layers.Dense(64, activation = tf.nn.relu, name="dense1")(inputs)
    x = tf.keras.layers.Dense(64, activation = tf.nn.relu, name="dense2")(x)
    outputs = tf.keras.layers.Dense(10, activation=tf.nn.softmax, name="predictions")(x)
    model = tf.keras.Model(inputs,outputs)
    return model
base_model()



training_data = tfds.load("fashion_mnist", split="train")
test_data = tfds.load("fashion_mnist",split="test")

def format_image(data):
    image = data["image"]
    image = tf.reshape(image, [-1])
    image = tf.cast(image, tf.float32)
    image = image/ 255.0

    return image, data["label"]


In [13]:
constant = tf.constant([[2,4,6,7,7],[2,3,5,4,6]])
print(tf.shape(constant))
print(tf.reshape(constant,[5,2]))

tf.Tensor([2 5], shape=(2,), dtype=int32)
tf.Tensor(
[[2 4]
 [6 7]
 [7 2]
 [3 5]
 [4 6]], shape=(5, 2), dtype=int32)


In [14]:
print(training_data.element_spec)
training_data = training_data.map(format_image)
test_data = test_data.map(format_image)
train = training_data.shuffle(buffer_size=1024).batch(64)
test = test_data.batch(64)
train.element_spec

{'image': TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}


(TensorSpec(shape=(None, 784), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

In [33]:
loss = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer =  tf.keras.optimizers.Adam()
print(optimizer)

In [24]:
model = base_model()
len(model.trainable_weights)

6

In [28]:
i = 0
for _ in train:
    i += 1
print(i)

938


In [38]:
train_acc_metrics = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metrics = tf.keras.metrics.SparseCategoricalAccuracy()
tf.keras.metrics.SparseCategoricalAccuracy()

In [36]:
def apply_gradient(optimizer, model, x, y):
    with tf.GradientTape() as tape:
        logits = model(x) # Do prediciton
        loss_value = loss(y_pred=logits, y_true=y) #Calculate Losses
    
    gradients = tape.gradient(loss_value, model.trainable_weights) # Calculate Gradients
    optimizer.apply_gradients(zip(gradients,model.trainable_weights)) # Tunning Losses
    
    return logits, loss_value
        

In [61]:
pb = tqdm(total=100,position=0,leave=True, bar_format= "{l_bar}{bar}| {n_fmt}/{total_fmt}")
for i in range(100):
    
    pb.set_description("Nyoba %d" % (i))
    pb.update()
    

Nyoba 99:  99%|█████████▉| 99/100

In [77]:
def train_data_one_epoch():
    losses = []
    pb = tqdm(total = len(list(enumerate(train))) ,position=0,leave=True, bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt}")
    for step, (x_batch_train,y_batch_train) in enumerate(train):

        logits, loss_value = apply_gradient(optimizer, model, x_batch_train, y_batch_train)
        losses.append(loss_value)
        train_acc_metrics(y_batch_train,logits)
        pb.set_description("Training Loss for Step %d: %.4f" % ( int(step),float(loss_value)))
        pb.update()
    return losses

def do_validation():
    losses = []
    for x_val ,y_val in test:
        
        val_logits = model(x_val)
        val_loss = loss(y_true=y_val, y_pred=val_logits)
        losses.append(val_loss)
        val_acc_metrics(y_val,val_logits)
    return losses


In [78]:
model = base_model()
EPOCHS = 10
epochs_val_losses , epochs_train_losses = [], []

for epoch in range(EPOCHS):
    print("Start of the epoch : %d" % (epoch))
    losses_train = train_data_one_epoch()
    train_acc = train_acc_metrics.result()
    
    losses_val = do_validation()
    val_acc = val_acc_metrics.result()
    #print(len(losses_train))#938 < TOtal Batches >
    losses_train_mean = np.mean(losses_train)
    losses_val_mean = np.mean(losses_val)
    epochs_val_losses.append(losses_val_mean)
    epochs_train_losses.append(losses_train_mean)
    
    print("\nEpoch %s : Train lossL %.4f Validation Loss : %.4f , Train acc : %.4f Val acc : %.4f" % (epoch, losses_train_mean, losses_val_mean,train_acc,val_acc))
    
    train_acc_metrics.reset_states()
    val_acc_metrics.reset_states()


Start of the epoch : 0


Training Loss for Step 937: 0.4369: 100%|█████████▉| 937/938


Epoch 0 : Train lossL 0.4811 Validation Loss : 0.4326 , Train acc : 0.8308 Val acc : 0.8359
Start of the epoch : 1


Training Loss for Step 937: 0.4369: 100%|██████████| 938/938
Training Loss for Step 937: 0.3722: 100%|██████████| 938/938



Epoch 1 : Train lossL 0.3619 Validation Loss : 0.3767 , Train acc : 0.8671 Val acc : 0.8612
Start of the epoch : 2


...


Epoch 7 : Train lossL 0.2627 Validation Loss : 0.3645 , Train acc : 0.9021 Val acc : 0.8714
Start of the epoch : 8


Training Loss for Step 937: 0.2183: 100%|██████████| 938/938
Training Loss for Step 937: 0.3560: 100%|█████████▉| 937/938


Epoch 8 : Train lossL 0.2538 Validation Loss : 0.3432 , Train acc : 0.9064 Val acc : 0.8755
Start of the epoch : 9


Training Loss for Step 937: 0.3560: 100%|██████████| 938/938
Training Loss for Step 937: 0.5484: 100%|██████████| 938/938


Epoch 9 : Train lossL 0.2475 Validation Loss : 0.3485 , Train acc : 0.9072 Val acc : 0.8778


In [79]:
%whos

Variable               Type                             Data/Info
-----------------------------------------------------------------
EPOCHS                 int                              10
apply_gradient         function                         <function apply_gradient at 0x00000284E57A6A60>
base_model             function                         <function base_model at 0x00000284E6B7EBF8>
constant               EagerTensor                      tf.Tensor(\n[[2 4 6 7 7]\<...>hape=(2, 5), dtype=int32)
do_validation          function                         <function do_validation at 0x00000284E57A6268>
epoch                  int                              9
epochs_train_losses    list                             n=10
epochs_val_losses      list                             n=10
format_image           function                         <function format_image at 0x00000284E6BF12F0>
i                      int                              99
loss                   SparseCategoricalCrossentr